In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
#yrs=np.arange(1983,2023,1)
#yrs=np.arange(1979,2021,1)
#yrs=np.arange(1997,2023,1)
yrs=np.arange(1991,2017,1)
#yrs=np.arange(1982,2021,1)
#yrs=np.arange(1979,2023,1)
#yrs=np.arange(2001,2023,1)
#yrs=np.arange(2001,2021,1)


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/'+'gpcp_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_1950-2016_QualityMask.nc')
    #quality=ds['p']
    ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
    precip=ds['p']#.where(quality==1)
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
    #precip=ds['precip'].transpose('time','lat','lon')
    
    #ds=xr.open_dataset(dir2+'precip.'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset('/Volumes/My Passport/day/imerg_combined_deres_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/'+'gpcp3_precip_'+str(year)+'.nc')
    #precip=ds['precip']
    
    if 1==0:#PERSIANN-CDR, CPC
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    
    if i==0:
        max_concat=defined1
        present_concat=defined
    else:
        max_concat=xr.concat([max_concat,defined1],dim='time')
        present_concat=xr.concat([present_concat,defined],dim='time')

In [ ]:
max_all=max_concat.sum(dim='time')
present_all=present_concat.sum(dim='time')
ratio_all=present_all/max_all

In [ ]:
#yrs=np.arange(1983,2023,1)
yrs=np.arange(1997,2021,1)
#yrs=np.arange(1997,2023,1)
#yrs=np.arange(1950,2017,1)
#yrs=np.arange(1982,2021,1)
#yrs=np.arange(1979,2023,1)
#yrs=np.arange(2001,2023,1)
#yrs=np.arange(2001,2021,1)
#yrs=np.arange(2001,2021,1)

#lats_sel=[x for x in ratio_all.lat.values if -60<=x<=70]
#ratio_all=ratio_all.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'persiann_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir2+'mswep_threshold_final_98_all_30.nc')
#ds=xr.open_dataset(dir_data+'mswep_threshold_r99p.nc')
#threshold=ds['threshold']#.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'gpcp_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'regen_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'gpcc_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'cpc_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir3+'imerg_threshold_final_98_all.nc')
#threshold=ds['threshold']

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    #precip=ds['precip']#.sel(lat=lats_sel)
    
    ds=xr.open_dataset(dir_data+'holding/'+'gpcp_precip_'+str(year)+'.nc')
    if i==0:
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    precip=ds['precip'].sel(lat=lats_sel)
    
    
    #if i==0:
    #    ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_1950-2016_QualityMask.nc')
    #    quality=ds['p']
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
    #precip=ds['p'].sel(lat=lats_sel)
    #precip=ds['p'].where(quality==1).sel(lat=lats_sel)
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].transpose('time','lat','lon').sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir_data+'precip.'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset('/Volumes/My Passport/day/imerg_combined_deres_'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/'+'gpcp3_precip_'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].sel(lat=lats_sel)
    
    
    #ds=xr.open_dataset(dir_data+'holding/merra2_precip_'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir_data+'holding/era_5_precip_'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir2+'jra55_precip_'+str(year)+'.nc')
    #if i==0:
    #    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].sel(lat=lats_sel)
    
    
    if 1==0:
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()

    if 1==0:
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold.sel(time=dt.datetime(1981,2,28))
                threshold_new2=threshold.sel(time=dt.datetime(1981,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold.sel(time=dt.datetime(1981,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values
    if 1==0:
        threshold_new=threshold
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    ones_sel=xr.ones_like(precip).where(precip>=1)
    #ones_sel=ones_sel.where(precip>=threshold_new)
    #freq_total=ones_sel.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all>=0.9)/ratio
    freq_total=ones_sel.sum(dim='time',skipna=True).where(ratio>=0.9)/ratio
    
    if i==0:
        freq_concat=freq_total
    else:
        freq_concat=xr.concat([freq_concat,freq_total],dim='time')

In [ ]:
#yrs=np.arange(1983,2023,1)
#yrs=np.arange(1979,2021,1)
#yrs=np.arange(1997,2023,1)
#yrs=np.arange(1950,2017,1)
#yrs=np.arange(1982,2021,1)
#yrs=np.arange(1979,2023,1)
yrs=np.arange(2001,2023,1)

#lats_sel=[x for x in ratio_all.lat.values if -60<=x<=70]
#ratio_all=ratio_all.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'persiann_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

ds=xr.open_dataset(dir2+'mswep_threshold_final_98_all_30.nc')
threshold=ds['threshold']#.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'gpcp_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'regen_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'gpcc_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'cpc_threshold_final_98_all.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir3+'imerg_threshold_final_98_all.nc')
#threshold=ds['threshold']

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir_data+'holding/'+'gpcp_'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
    #precip=ds['p'].where(quality==1).sel(lat=lats_sel)
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
    #precip=ds['precip'].transpose('time','lat','lon').sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir_data+'precip.'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    ds=xr.open_dataset('/Volumes/My Passport/day/imerg_combined_deres_'+str(year)+'.nc')
    lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    precip=ds['precip'].sel(lat=lats_sel)
    
    if 1==1:
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()

    if 1==1:
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold.sel(time=dt.datetime(2001,2,28))
                threshold_new2=threshold.sel(time=dt.datetime(2001,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold.sel(time=dt.datetime(2001,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    ones_sel=xr.ones_like(precip).where(precip>=1)
    ones_sel=ones_sel.where(precip>=threshold_new)
    freq_total=ones_sel.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all>=0.9)/ratio
    #freq_total=ones_sel.sum(dim='time',skipna=True).where(ratio>=0.9)/ratio
    
    if i==0:
        freq_concat1=freq_total
    else:
        freq_concat1=xr.concat([freq_concat1,freq_total],dim='time')

In [ ]:
diff=freq_concat1-freq_concat

In [ ]:
mean=freq_concat.mean(dim='time',skipna=True)
field=mean

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
#cmap1=plt.cm.viridis(np.linspace(0,1,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=300,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#300#13#2.6
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(50,350,50))
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(2,14,2))
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15,ticks=np.arange(0.5,3,0.5))
#ax.contour(field.lon,field.lat,field,levels=np.arange(50,350,50),colors='k',linewidths=2.0)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('days yr$^{-1}$',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')#,linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':22,'color':'k'}#24
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.bottom_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-90,90)       

#ax.set_title(r'$\bf{MSWEP}$'+' '+r'$\bf{V2.8}$'+' '+r'$\bf{(1991-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{GPCP}$'+' '+r'$\bf{with}$'+' '+r'$\bf{adjustment}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{GPCP}$'+' '+r'$\bf{v01r03}$'+' '+r'$\bf{(1997-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{REGEN}$'+' '+r'$\bf{HQ}$'+' '+r'$\bf{(1991-2016)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{REGEN}$'+' '+r'$\bf{(1991-2016)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{GPCC}$'+' '+r'$\bf{V2022}$'+' '+r'$\bf{(1991-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{IMERG}$'+' '+r'$\bf{V07a}$'+' '+r'$\bf{(2001-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{GPCP}$'+' '+r'$\bf{V3.2}$'+' '+r'$\bf{(2001-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{CPC}$'+' '+r'$\bf{Unified}$'+' '+r'$\bf{(1991-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{(1991-2020)}$',fontsize=34,pad=5)
ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{(1991-2020)}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{(1991-2020)}$',fontsize=34,pad=5)



#ax.set_title(r'$\bf{F1mm}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{F98p_{all}}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{F99p}$',fontsize=34,pad=5)

#ax.set_title('Wet day total',fontsize=46,pad=5)
#ax.set_title('EP total',fontsize=46,pad=5)
plt.show()

In [ ]:
#fig.savefig(dir_data+'wet_days_paper_4.png')
fig.savefig(dir_data+'wet_days_paper_44a.png')

In [ ]:
mean=diff.mean(dim='time',skipna=True)
field=mean

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-3,vmax=3,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=15)
#ax.contour(field.lon,field.lat,field,levels=np.arange(50,350,50),colors='k',linewidths=2.0)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('days yr$^{-1}$',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
            
#ax.set_title(r'$\bf{PERSIANN-CDR}$'+' '+r'$\bf{(1983-2022)}$',fontsize=34,pad=5)
ax.set_title(r'$\bf{Difference}$',fontsize=34,pad=5)
#ax.set_title('Wet day total',fontsize=46,pad=5)
#ax.set_title('EP total',fontsize=46,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'wet_days_2c.png')